In [1]:
!wget "https://github.com/krutika-shimpi/MyRepository/raw/main/diabetes.csv"

--2024-11-28 16:36:53--  https://github.com/krutika-shimpi/MyRepository/raw/main/diabetes.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/krutika-shimpi/MyRepository/main/diabetes.csv [following]
--2024-11-28 16:36:53--  https://raw.githubusercontent.com/krutika-shimpi/MyRepository/main/diabetes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23105 (23K) [text/plain]
Saving to: ‘diabetes.csv’

diabetes.csv        100%[===================>]  22.56K  --.-KB/s    in 0.02s   

2024-11-28 16:36:54 (1.46 MB/s) - ‘diabetes.csv’ saved [23105/23105]



In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
data = pd.read_csv("diabetes.csv")
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
data.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [5]:
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_scaled = sc.fit_transform(X)

In [7]:
from sklearn.model_selection import train_test_split as split
X_train, X_test, y_train, y_test = split(X_scaled, y, test_size = 0.2, random_state = 42)

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input

model = Sequential([
    Input(shape = [8,], ),
    Dense(32, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])


model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = ['accuracy'])

history = model.fit(X_train, y_train, epochs = 100, batch_size = 32, validation_data = (X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6227 - loss: 0.6623 - val_accuracy: 0.6753 - val_loss: 0.6269
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7141 - loss: 0.5928 - val_accuracy: 0.7208 - val_loss: 0.5861
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7385 - loss: 0.5644 - val_accuracy: 0.7403 - val_loss: 0.5596
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7343 - loss: 0.5480 - val_accuracy: 0.7403 - val_loss: 0.5439
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7455 - loss: 0.5151 - val_accuracy: 0.7597 - val_loss: 0.5305
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7691 - loss: 0.5113 - val_accuracy: 0.7597 - val_loss: 0.5232
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7734 - loss: 0.4797 - val_accuracy: 0.7662 - val_loss: 0.5167
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7735 - loss: 0.4820 - val_accuracy: 0.7597 - 

In [9]:
!pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.6 MB/s eta 0:00:00


### How to select best optimizer

In [14]:
import keras_tuner as kt

def build_model(hp):

  model = Sequential([
      Input(shape = [8,]),
      Dense(32, activation = 'relu'),
      Dense(1, activation = 'sigmoid')
  ])

  optimizer = hp.Choice('optimizers', values = ['sgd', 'adam', 'rmsprop', 'adadelta'])

  model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5)

In [ ]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizers': 'rmsprop'}

In [ ]:
model = tuner.get_best_models(num_models = 1)[0]

In [ ]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100, initial_epoch = 6, validation_data = (X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7602 - loss: 0.5226 - val_accuracy: 0.7792 - val_loss: 0.5171
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7844 - loss: 0.4923 - val_accuracy: 0.7857 - val_loss: 0.5102
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7717 - loss: 0.4829 - val_accuracy: 0.7792 - val_loss: 0.5076
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7674 - loss: 0.4757 - val_accuracy: 0.7662 - val_loss: 0.5055
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7648 - loss: 0.4787 - val_accuracy: 0.7662 - val_loss: 0.5031
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7751 - loss: 0.4599 - val_accuracy: 0.7727 - val_loss: 0.5034
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7929 - loss: 0.4597 - val_accuracy: 0.7727 - val_loss: 0.5034
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7876 - loss: 0.4455 - val_accuracy: 0.76

In [ ]:
def build_model(hp):

  units = hp.Int(8, 128, step = 8)
  model = Sequential([
      Input(shape = [8,]),
      Dense(units, activation = 'relu'),
      Dense(1, activation = 'sigmoid')
  ])


  optimizer = hp.Choice('optimizers', values = ['sgd', 'adam', 'rmsprop', 'adadelta'])

  model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

In [24]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units', 8, 128, step = 8)
  model.add(Input(shape = [8,]))
  for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
    model.add(Dense(units, activation = 'relu'))
  model.add(Dense(1, activation = 'sigmoid'))

  optimizer = hp.Choice('optimizers', values = ['sgd', 'adam', 'rmsprop', 'adadelta'])
  model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [25]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5)

Reloading Tuner from ./untitled_project/tuner0.json


In [26]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.5974025726318359

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 08m 25s


In [28]:
model = tuner.get_best_models(num_models = 1)[0]

In [31]:
model.fit(X_train, y_train, epochs = 100, initial_epoch = 6, validation_data = (X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.7563 - loss: 0.4941 - val_accuracy: 0.7273 - val_loss: 0.5638
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8087 - loss: 0.4394 - val_accuracy: 0.7338 - val_loss: 0.6049
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8234 - loss: 0.3740 - val_accuracy: 0.7013 - val_loss: 0.5751
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8572 - loss: 0.3665 - val_accuracy: 0.7143 - val_loss: 0.5815
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8443 - loss: 0.3582 - val_accuracy: 0.6753 - val_loss: 0.6191
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8350 - loss: 0.3494 - val_accuracy: 0.7468 - val_loss: 0.6779
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8611 - loss: 0.3134 - val_accuracy: 0.6883 - val_loss: 0.6684
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8773 - loss: 0.2728 - val_accuracy: 0.65